In [232]:
using CSV, DataFrames, Statistics, Random, Plots, Distances, Clustering, Gurobi, JuMP,  Statistics, Dates
using ScikitLearn, DecisionTree, MLDataUtils, CategoricalArrays

In [224]:
aapl = CSV.read("AAPL.csv", DataFrame)
tsla = CSV.read("TSLA.csv", DataFrame)

# Using AVG daily Price
#aapl_price = (aapl[!, :High] + aapl[!, :Low] ) / 2
#tsla_price = (tsla[!, :High] + tsla[!, :Low] ) / 2

# Using Close Price 
aapl_price = aapl[!, :Close] 
tsla_price = tsla[!, :Close] 

aapl_subset = aapl_price[1:500]
tsla_subset = tsla_price[1:250]

aapl_df_subset = first(aapl, 500)

A = aapl_subset

#A = [aapl_subset]


# Assuming the data for AAPL and TSLA is aligned by date,
# construct the A and B matrices
#A = [aapl_price tsla_price];


Initial_Cash = 1000


1000

In [221]:
print(aapl_subset)

[44.645, 46.205002, 44.172501, 43.68, 42.122501, 42.400002, 42.157501, 42.275002, 42.737499, 41.369999, 40.985001, 41.517502, 40.2225, 39.2075, 37.682499, 36.7075, 39.2925, 39.037498, 39.057499, 39.435001, 39.48, 35.547501, 37.064999, 36.982498, 37.6875, 38.327499, 38.450001, 38.072498, 37.5, 38.267502, 38.735001, 38.965, 39.205002, 38.325001, 38.48, 38.174999, 39.439999, 39.075001, 38.669998, 41.3125, 41.610001, 41.630001, 42.8125, 43.544998, 43.560001, 42.735001, 42.602501, 42.357498, 42.7225, 42.544998, 42.700001, 42.605, 42.732498, 43.0075, 42.764999, 43.2425, 43.557499, 43.5825, 43.717499, 43.287498, 43.7425, 43.962502, 43.8825, 43.630001, 43.125, 43.227501, 44.724998, 45.227501, 45.427502, 45.932499, 46.529999, 47.005001, 46.6325, 47.040001, 48.772499, 47.762501, 47.185001, 46.697498, 47.1175, 47.18, 47.487499, 47.810001, 48.505001, 48.837502, 48.922501, 49.25, 50.025002, 49.875, 50.154999, 49.737499, 49.717499, 49.807499, 49.8125, 50.782501, 50.965, 51.1325, 51.869999, 51.790001

In [211]:
#model = Model(Gurobi.Optimizer)

In [212]:
num_days = size(A,1)

500

In [213]:
num_stocks = size(A, 2)

1

In [214]:
if num_stocks > 1
    model_mult_stocks = Model(Gurobi.Optimizer);

    # VARIABLES TO ONLY ALLOW SELLING 1 or BUYING 1 STOCK EACH DAY
    @variable(model_mult_stocks, x[1:num_days, 1:num_stocks] , Bin) ;
    @variable(model_mult_stocks, y[1:num_days, 1:num_stocks] , Bin) ;

    # VARIABLES TO ALLOW BUYING / SELLING UP TO 'n' STOCKS EACH DAY
    #@variable(model_mult_stocks, x[1:num_days, 1:num_stocks] , Int, lower_bound = 0, upper_bound = 3)
    #@variable(model_mult_stocks, y[1:num_days, 1:num_stocks] , Int, lower_bound = 0, upper_bound = 3) 

    @variable(model_mult_stocks, C[1:num_days] >= 0);
    @variable(model_mult_stocks, S[1:num_days, 1:num_stocks] >= 0);


    # Objective to maximize total asset value = $1799.1 for this problem 
    @objective(model_mult_stocks, Max, sum(S[num_days, i] * A[num_days, i] for i in 1:num_stocks) + C[num_days]);

    # Objective to Maximize final cash (i.e will want to sell everything) Results in 1646.91 
    #@objective(model, Max, C[num_days]);


    # Initial conditions
    @constraint(model_mult_stocks, C[1] == Initial_Cash);
    @constraint(model_mult_stocks, [i=1:num_stocks], S[1,i] == 0);

    # UPDATED CASH UPDATE TO FIX BUG 
    @constraint(model_mult_stocks, C[2] == C[1] - sum(x[1, i] * A[1, i] * 1.05 for i in 1:num_stocks));
    @constraint(model_mult_stocks, [t=3:num_days], C[t] == C[t-1] - sum(x[t-1, i] * A[t-1, i] * 1.05 for i in 1:num_stocks) + sum(y[t-2, i] * A[t-2, i] * .95 for i in 1:num_stocks));

    # Cash and stock update
    @constraint(model_mult_stocks, [t=2:num_days, i=1:num_stocks], S[t,i] == S[t-1,i] + x[t-1,i] - y[t-1,i]);

    # Buy-sell constraints
    @constraint(model_mult_stocks, [t=1:num_days, i=1:num_stocks], x[t,i] * y[t,i] == 0);

    # Budget and asset availability
    @constraint(model_mult_stocks, [t=1:num_days], sum(x[t, i] * A[t, i] * 1.05 for i in 1:num_stocks) <= C[t]);
    @constraint(model_mult_stocks, [t=1:num_days, i=1:num_stocks], y[t,i] <= S[t,i]);
else 
    
    model_sing_stock = Model(Gurobi.Optimizer)
    # VARIABLES TO ONLY ALLOW SELLING 1 or BUYING 1 STOCK EACH DAY
    @variable(model_sing_stock, x[1:num_days] , Bin) ;
    @variable(model_sing_stock, y[1:num_days] , Bin) ;

    # VARIABLES TO ALLOW BUYING / SELLING UP TO 'n' STOCKS EACH DAY
    #@variable(model_sing_stock, x[1:num_days] , Int, lower_bound = 0, upper_bound = 3)
    #@variable(model_sing_stock, y[1:num_days] , Int, lower_bound = 0, upper_bound = 3) 

    @variable(model_sing_stock, C[1:num_days] >= 0);
    @variable(model_sing_stock, S[1:num_days] >= 0);
    
    
    @objective(model_sing_stock, Max, sum(S[num_days] * A[num_days]) + C[num_days]);
    
    
    # Initial conditions
    @constraint(model_sing_stock, C[1] == Initial_Cash);
    @constraint(model_sing_stock,  S[1] == 0);

    # UPDATED CASH UPDATE TO FIX BUG 
    @constraint(model_sing_stock, C[2] == C[1] - sum(x[1] * A[1] * 1.05));
    @constraint(model_sing_stock, [t=3:num_days], C[t] == C[t-1] - sum(x[t-1] * A[t-1] * 1.05) + sum(y[t-2] * A[t-2] * .95 ));
    # Cash and stock update
    @constraint(model_sing_stock, [t=2:num_days], S[t] == S[t-1] + x[t-1] - y[t-1]);

    # Buy-sell constraints
    @constraint(model_sing_stock, [t=1:num_days], x[t] * y[t] == 0);

    # Budget and asset availability
    @constraint(model_sing_stock, [t=1:num_days], sum(x[t] * A[t] * 1.05) <= C[t]);
    @constraint(model_sing_stock, [t=1:num_days], y[t] <= S[t]);
end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31


500-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 y[1] - S[1] ≤ 0
 y[2] - S[2] ≤ 0
 y[3] - S[3] ≤ 0
 y[4] - S[4] ≤ 0
 y[5] - S[5] ≤ 0
 y[6] - S[6] ≤ 0
 y[7] - S[7] ≤ 0
 y[8] - S[8] ≤ 0
 y[9] - S[9] ≤ 0
 y[10] - S[10] ≤ 0
 y[11] - S[11] ≤ 0
 y[12] - S[12] ≤ 0
 y[13] - S[13] ≤ 0
 ⋮
 y[489] - S[489] ≤ 0
 y[490] - S[490] ≤ 0
 y[491] - S[491] ≤ 0
 y[492] - S[492] ≤ 0
 y[493] - S[493] ≤ 0
 y[494] - S[494] ≤ 0
 y[495] - S[495] ≤ 0
 y[496] - S[496] ≤ 0
 y[497] - S[497] ≤ 0
 y[498] - S[498] ≤ 0
 y[499] - S[499] ≤ 0
 y[500] - S[500] ≤ 0

In [215]:
#set_optimizer_attribute(model, "MIPGap", 0.001) # Sets the gap to 5%

optimize!(model_sing_stock)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2000 rows, 2000 columns and 5993 nonzeros
Model fingerprint: 0x4d2b3db4
Model has 500 quadratic constraints
Variable types: 1000 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 106 rows and 564 columns
Presolve time: 0.02s
Presolved: 1894 rows, 1436 columns, 6566 nonzeros
Variable types: 232 continuous, 1204 integer (965 binary)

Root relaxation: objective 3.361367e+03, 160 iterations, 0.01 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3361.36651    0  192    

In [216]:
x_val = value.(x)
y_val = value.(y)
C_val = value.(C)
S_val = value.(S)

#rounded_c = round
println(typeof(x_val))
# Example of how to display these values
println("\n\nOptimal values of x: ", (x_val))
println("\n\nOptimal values of y: ", y_val)
println("\n\nOptimal values of cash (C): ", round.(C_val, digits = 2))
println("\n\nOptimal stock holdings (S): ", S_val)

Vector{Float64}


Optimal values of x: [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0

Optimal stock holdings (S): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 22.0, 23.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0, 18.0, 17.0, 16.0, 15.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 15.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0,

In [217]:
# Assuming the model has been solved and S_val has been obtained

# If S_val is structured with days in rows and stocks in columns
final_stock_holdings = value.(S[end, :])  # Extracting the final stock holdings for each stock

# If A is structured with days in rows and stocks in columns
final_stock_prices = A[end, :]            # Extracting the final stock prices

# Ensure both arrays have the same length
if length(final_stock_holdings) == length(final_stock_prices)
    # Calculating the total portfolio value
    total_portfolio_value = round.(sum(final_stock_holdings .* final_stock_prices) + value(C[end]), digits = 2)
    println("Total value of the portfolio at the end of the period: \$", total_portfolio_value)
else
    println("Mismatch in dimensions of stock holdings and stock prices arrays")
end

Total value of the portfolio at the end of the period: $3319.26


In [218]:
# Calculate total value of stock holdings
total_stock_value = sum(final_stock_holdings .* final_stock_prices)

# Extract final cash balance
final_cash_balance = value(C[end])

# Calculate total portfolio value
total_portfolio_value = total_stock_value + final_cash_balance

println("Total value of the portfolio (stocks) at the end of the period: \$", round.(total_stock_value, digits = 2))

    println("Total value of Cash at the end of the period: \$ ", round.(final_cash_balance, digits = 2))


Total value of the portfolio (stocks) at the end of the period: $2648.91
Total value of Cash at the end of the period: $ 670.35
